In [10]:
!pip install Flask Flask-Dance[google] Flask-Dropzone openai dropbox

In [ ]:
from flask import Flask, render_template, request, flash, redirect, url_for, session
from flask_dance.contrib.google import make_google_blueprint, google
from flask_dropzone import Dropzone
import openai
import dropbox
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['UPLOAD_PATH'] = 'uploads'
app.config['DROPBOX_ACCESS_TOKEN'] = 'your_dropbox_access_token'
app.config['OPENAI_API_KEY'] = 'your_openai_api_key'

# Flask-Dance Google Blueprint
google_bp = make_google_blueprint(client_id='your_google_client_id', 
                                 client_secret='your_google_client_secret',
                                 redirect_to='google_login')

app.register_blueprint(google_bp, url_prefix='/google_login')

# Flask-Dropzone
dropzone = Dropzone(app)

# Set OpenAI API key
openai.api_key = app.config['OPENAI_API_KEY']

# Configure Dropbox
dbx = dropbox.Dropbox(app.config['DROPBOX_ACCESS_TOKEN'])

def generate_chatgpt_response(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200
    )
    return response.choices[0].text.strip()

def upload_to_dropbox(file_path, file_name):
    with open(file_path, 'rb') as f:
        dbx.files_upload(f.read(), '/' + file_name, mode=dropbox.files.WriteMode('overwrite'))

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/google_login')
def google_login():
    if not google.authorized:
        return redirect(url_for('google.login'))
    resp = google.get('/plus/v1/people/me')
    assert resp.ok, resp.text
    session['user_email'] = resp.json()['emails'][0]['value']
    flash(f'Logged in as {session["user_email"]}', 'success')
    return redirect(url_for('index'))

@app.route('/logout')
def logout():
    session.pop('user_email', None)
    flash('Logged out', 'success')
    return redirect(url_for('index'))

@app.route('/analyze', methods=['GET', 'POST'])
def analyze():
    if 'user_email' not in session:
        return redirect(url_for('index'))

    if request.method == 'POST':
        # Retrieve uploaded file and proposal details
        uploaded_file = request.files.get('file')
        proposal_details = request.form.get('proposal_details')

        # Save the uploaded file
        if uploaded_file:
            file_path = os.path.join(app.config['UPLOAD_PATH'], uploaded_file.filename)
            uploaded_file.save(file_path)

            # Upload to Dropbox
            upload_to_dropbox(file_path, uploaded_file.filename)

            # Perform ChatGPT analysis
            chatgpt_response = generate_chatgpt_response(proposal_details)

            # Display the result or save it in the database

            flash('Analysis completed successfully!', 'success')
            return render_template('analyze.html', chatgpt_response=chatgpt_response)

    return render_template('analyze.html')

if __name__ == '__main__':
    if not os.path.exists(app.config['UPLOAD_PATH']):
        os.makedirs(app.config['UPLOAD_PATH'])
    app.run(debug=True)
